In [1]:
import os
os.chdir('/workspace')
import pandas as pd
import matplotlib.pyplot as plt

import os

# 현재 위치
os.getcwd()

'/workspace'

In [2]:
from backtesting.test import EURUSD,GOOG

data_path="/workspace/data/HBARUSDT_hour.csv"
stock=pd.read_csv(data_path)
time= pd.to_datetime(stock['open_time'], unit='ms')
stock["time"]=time
stock.set_index('time', inplace=True)


/opt/conda/lib/python3.11/site-packages/backtesting/test/__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),
/opt/conda/lib/python3.11/site-packages/backtesting/test/__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),


In [11]:
from backtesting import Strategy, Backtest
from backtesting.lib import SignalStrategy, TrailingStrategy
from backtesting.lib import crossover
from backtesting.test import SMA


class MovingAverageStrategy(Strategy):
    short_window = 100  # 10-day moving average

    def init(self):
        # Calculate 10-day Simple Moving Average (SMA)
        self.sma = self.I(SMA, self.data.Close, self.short_window)

    def next(self):
        # If price crosses above SMA, buy
        if crossover(self.data.Close, self.sma):
            self.buy()
        # If price crosses below SMA, sell
        elif crossover(self.sma, self.data.Close):
            self.position.close()
            
            

backtest = Backtest(stock, MovingAverageStrategy,
              cash=10000,
              commission=0.0002,
              exclusive_orders=True)

output=backtest.run()
#output.to_csv("/workspace/result/Mamba2_S.csv")
output

Start                     2019-09-29 04:00:00
End                       2024-12-25 13:00:00
Duration                   1914 days 09:00:00
Exposure Time [%]                   49.679808
Equity Final [$]                  3713.783104
Equity Peak [$]                  35918.165281
Return [%]                         -62.862169
Buy & Hold Return [%]              687.038909
Return (Ann.) [%]                  -17.204486
Volatility (Ann.) [%]              130.096795
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -97.139776
Avg. Drawdown [%]                  -20.001853
Max. Drawdown Duration     1777 days 21:00:00
Avg. Drawdown Duration      173 days 15:00:00
# Trades                                 1248
Win Rate [%]                        14.903846
Best Trade [%]                     165.233736
Worst Trade [%]                    -13.913998
Avg. Trade [%]                    

In [12]:
class SmaCross(SignalStrategy,
               TrailingStrategy):
    n1 = 10
    n2 = 25
    
    def init(self):
        # In init() and in next() it is important to call the
        # super method to properly initialize the parent classes
        super().init()
        
        # Precompute the two moving averages
        sma1 = self.I(SMA, self.data.Close, self.n1)
        sma2 = self.I(SMA, self.data.Close, self.n2)
        
        # Where sma1 crosses sma2 upwards. Diff gives us [-1,0, *1*]
        signal = (pd.Series(sma1) > sma2).astype(int).diff().fillna(0)
        signal = signal.replace(-1, 0)  # Upwards/long only
        
        # Use 95% of available liquidity (at the time) on each order.
        # (Leaving a value of 1. would instead buy a single share.)
        entry_size = signal * .95
                
        # Set order entry sizes using the method provided by 
        # `SignalStrategy`. See the docs.
        self.set_signal(entry_size=entry_size)
        
        # Set trailing stop-loss to 2x ATR using
        # the method provided by `TrailingStrategy`
        self.set_trailing_sl(2)
        
backtest = Backtest(stock, SmaCross,
              cash=10000,
              commission=0.0002,
              exclusive_orders=True)

output=backtest.run()
#output.to_csv("/workspace/result/Mamba2_S.csv")
output   

Start                     2019-09-29 04:00:00
End                       2024-12-25 13:00:00
Duration                   1914 days 09:00:00
Exposure Time [%]                   35.933348
Equity Final [$]                  2630.883896
Equity Peak [$]                  12722.772327
Return [%]                         -73.691161
Buy & Hold Return [%]              687.038909
Return (Ann.) [%]                  -22.469769
Volatility (Ann.) [%]               55.136315
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -94.324337
Avg. Drawdown [%]                  -28.423627
Max. Drawdown Duration     1778 days 23:00:00
Avg. Drawdown Duration      382 days 12:00:00
# Trades                                 1050
Win Rate [%]                        39.428571
Best Trade [%]                       64.36214
Worst Trade [%]                    -17.789186
Avg. Trade [%]                    